# Hosting OLS (EMBL-EBI Ontology Lookup Service) MCP Server on Amazon Bedrock AgentCore Runtime 

## Overview

In this tutorial we will learn how to host OLS (EMBL-EBI Ontology Lookup Service) MCP Server  on Amazon Bedrock AgentCore Runtime. We will use the Amazon Bedrock AgentCore Python SDK to wrap MCP tools as an MCP server compatible with Amazon Bedrock AgentCore.


### Tutorial Details

| Information         | Details                                                   |
|:--------------------|:----------------------------------------------------------|
| Tutorial type       | Hosting Tools                                             |
| Tool type           | MCP server                                                |
| Tutorial components | Hosting MCP server on AgentCore Runtime                  |
| Use case             | Data enrichment and ontology mapping                                          |
| Example complexity  | Easy                                                      |
| SDK used            | Amazon BedrockAgentCore Python SDK and MCP               |



## Prerequisites

This notebook depends upon an existing Cognito credentials provider setup and will retrieve its details from the parameter store with prefix'/app/researchapp/agentcore/'. 

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials configured
* Amazon Bedrock AgentCore SDK
* MCP (Model Context Protocol) library

## Start by cloning the OLS MCP server for https://www.ebi.ac.uk/ols4/


In [ ]:
!git clone https://github.com/seandavi/ols-mcp-server.git ols

In [ ]:
!uv sync --directory ols/

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from boto3.session import Session
from pathlib import Path
import os
import sys

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])



In [ ]:
boto_session = Session()
region = boto_session.region_name

ssm_client = boto_session.client('ssm', region_name=region)
secrets_client = boto_session.client('secretsmanager', region_name=region)
agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)
ssm_client = boto_session.client('ssm', region_name=region)

tool_name = "ols_mcp_server_agentcore"

## Understanding MCP (Model Context Protocol)

MCP is a protocol that allows AI models to securely access external data and tools. Key concepts:

* **Tools**: Functions that the AI can call to perform actions
* **Streamable HTTP**: Transport protocol used by AgentCore Runtime
* **Session Isolation**: Each client gets isolated sessions via `Mcp-Session-Id` header
* **Stateless Operation**: Servers must support stateless operation for scalability

AgentCore Runtime expects MCP servers to be hosted on `0.0.0.0:8000/mcp` as the default path.

### Project Structure

Let's set up our project with the proper structure:

```
ols/src/ols_mcp_server/
├── server.py              # Main MCP server code
├── models.py               # data models
├── requirements.txt          # Dependencies
└── __init__.py              # Python package marker
```

## Updating MCP Server

Now let's update the MCP server to use FastMCP with `stateless_http=True` which is required for AgentCore Runtime compatibility and Run in http mode instead of the default. 

In [10]:
#Make the following code updates directly in 'ols/src/ols_mcp_server/server.py'

# Initialize the MCP server with stateless_http=True
#mcp = FastMCP(host="0.0.0.0", stateless_http=True)

# Run in http mode instead
#mcp.run(transport="streamable-http")

## Creating Local Testing Client

Before deploying to AgentCore Runtime, let's create a client to test our MCP server locally:

In [ ]:
%%writefile my_mcp_client.py
import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

### Testing Locally

To test your MCP server locally:

1. **Terminal 1**: Start the MCP server
   ```bash
   python mcp_server.py
   ```
   
2. **Terminal 2**: Run the test client
   ```bash
   python my_mcp_client.py
   ```

You should see your three tools listed in the output.

## Use existing Amazon Cognito for Authentication

AgentCore Runtime requires authentication. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server.

In [ ]:
print("Use existing  up Amazon Cognito user pool...")
from utils import get_ssm_parameter
runtime_client_id = get_ssm_parameter("/app/researchapp/agentcore/machine_client_id")
runtime_client_secret = get_ssm_parameter("/app/researchapp/agentcore/cognito_secret")
runtime_cognito_discovery_url = get_ssm_parameter("/app/researchapp/agentcore/cognito_discovery_url")
runtime_user_pool_id = get_ssm_parameter("/app/researchapp/agentcore/userpool_id")
cognito_config = { "pool_id": runtime_user_pool_id,
            "client_id": runtime_client_id,
            "discovery_url": runtime_cognito_discovery_url }
print(f"User Pool ID: {cognito_config.get('pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

Add dependencies of OLS to a requirements.txt. Make sure there are no escape characters added. 

In [ ]:
!uv pip compile ols/pyproject.toml --no-color >> requirements.txt

## Configuring AgentCore Runtime Deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
print(f"Using AWS region: {region}")

required_files = ['ols/src/ols_mcp_server/server.py','ols/src/ols_mcp_server/models.py','ols/src/ols_mcp_server/__init__.py','requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="ols/src/ols_mcp_server/server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=tool_name+'1'
)
print("Configuration completed ✓")

## Launching MCP Server to AgentCore Runtime

Now that we've got a docker file, let's launch the MCP server to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

## Storing Configuration for Remote Access

Before we can invoke our deployed MCP server, let's store the Agent ARN  in AWS Systems Manager Parameter Store  for easy retrieval:

In [ ]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)

agent_arn_response = ssm_client.put_parameter(
    Name='/ols_mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

## Creating Remote Testing Client

Now let's create a client to test our deployed MCP server. This client will retrieve the necessary credentials from AWS and connect to the deployed server:

In [ ]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
import base64
import time
from boto3.session import Session
from datetime import timedelta
import traceback

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from utils import get_access_token


async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/ols_mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")      
        # Validate and refresh token if needed
        bearer_token = get_access_token()
        print(bearer_token)
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## Testing Your Deployed MCP Server

Let's test our deployed MCP server using the remote client:

In [ ]:
print("Testing deployed MCP server...")

!python my_mcp_client_remote.py

## Invoking MCP Tools Remotely

Now let's create an enhanced client that not only lists tools but also invokes them to demonstrate the full MCP functionality:

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
import base64
import time
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from utils import get_access_token


async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/ols_mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        
        
        # Validate and refresh token if needed
        bearer_token = get_access_token()
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                
                try:
                    print("\n➕ Testing search_terms(5, 3)...")
                    add_result = await session.call_tool(
                        name="search_terms",
                        arguments={"query": "MI"}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n✖️  Testing get_ontology_info(go)...")
                    multiply_result = await session.call_tool(
                        name="get_ontology_info",
                        arguments={"ontology_id": 'go'}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n👋 Testing search_ontologies('disease')...")
                    greet_result = await session.call_tool(
                        name="search_ontologies",
                        arguments={"search": "disease"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                print("\n✅ MCP tool testing completed!")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## Test Tool Invocation

Let's test our MCP tools by actually invoking them:

In [ ]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

## Connect the MCP server to your Terminology agent
First get the MCP server url and headers 

In [ ]:
!pip install pyyaml requests boto3

from boto3.session import Session
import boto3
import sys
import json
import os


from utils import get_access_token

boto_session = Session()
region = boto_session.region_name
    
print(f"Using AWS region: {region}")
    
    
try:
    ssm_client = boto3.client('ssm', region_name=region)
    agent_arn_response = ssm_client.get_parameter(Name='/ols_mcp_server/runtime/agent_arn')
    agent_arn = agent_arn_response['Parameter']['Value']
    print(f"Retrieved Agent ARN: {agent_arn}")

    # Validate and refresh token if needed
    bearer_token = get_access_token()
    
        
except Exception as e:
    print(f"Error retrieving credentials: {e}")
    sys.exit(1)
    
encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
print(f"\nmcp url: {mcp_url}")



Define the system prompt for the terminology agent

In [ ]:
SYSTEM_PROMPT="""
You are a **Medical Terminology Standardization Agent** with access to the EBI Ontology Lookup Service (OLS) through specialized tools. Your primary role is to standardize, disambiguate, and enrich medical terminology in user queries before they are processed by downstream applications.

## Core Responsibilities

### 1. **Terminology Standardization**
- Convert variant medical terms to official ontology identifiers
- Map "MI", "heart attack", "myocardial infarction" → MONDO:0005068
- Standardize "DM", "diabetes", "diabetic" → MONDO:0005015 (diabetes mellitus)
- Replace colloquial terms with precise scientific terminology

### 2. **Query Disambiguation** 
- Resolve ambiguous medical terms using ontological context
- Distinguish between different meanings of the same term
- Provide clear disambiguation notes for variant interpretations
- Choose the most clinically relevant ontology match

### 3. **Concept Expansion**
- Discover related terms and hierarchical relationships
- Find parent/child terms in ontological hierarchies
- Identify synonyms and alternative terminology
- Expand single terms to comprehensive concept networks

### 4. **Metadata Enrichment**
- Generate structured metadata for downstream applications
- Provide ontology IDs, IRIs, and relationship mappings
- Create standardized term dictionaries for consistent usage
- Format enriched data for multiple application types

## Available Tools

You have access to these EBI OLS tools:

- **search_terms**: Search across 200+ biological/medical ontologies
- **get_ontology_info**: Retrieve detailed ontology information  
- **search_ontologies**: Discover available ontologies by domain
- **get_term_info**: Get comprehensive details about specific terms
- **get_term_children**: Find direct child terms in hierarchies
- **get_term_ancestors**: Retrieve parent terms and ancestors
- **find_similar_terms**: Discover semantically similar terms

## Key Ontologies to Prioritize

1. **MONDO** - Disease classification and medical conditions
2. **HPO** - Human phenotypes for clinical descriptions
3. **GO** - Gene functions and biological processes  
4. **ChEBI** - Chemical compounds and drug substances
5. **EFO** - Experimental factors and biomedical concepts

## Response Format

Always structure your responses as:

```json
{
  "original_query": "user's original text",
  "standardized_terms": [
    {
      "original": "variant term",
      "standard": "official term", 
      "ontology_id": "MONDO:0005068",
      "ontology": "mondo",
      "confidence": "high|medium|low"
    }
  ],
  "expanded_concepts": ["related term 1", "related term 2"],
  "disambiguation_notes": ["clarification 1", "clarification 2"],
  "enrichment_metadata": {
    "terms_processed": 3,
    "ontologies_used": ["mondo", "hp"],
    "relationships_found": 5
  }
}
```

Remember: You are the **authoritative gateway** for medical terminology standardization. Downstream applications depend on your accuracy and consistency for proper functioning."""

Create and test the agent with the MCP client and Bedrock model

In [ ]:

from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
import time

def get_all_mcp_tools_from_mcp_client(client):
    """Get all tools from MCP client with pagination."""
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools



print("\n✅ Agent testing now!")
    # Create MCP client
client = MCPClient(
        lambda: streamablehttp_client(
            mcp_url, headers
        )
)
    
    # Create Bedrock model
model = BedrockModel(
        model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
        temperature=0.7,
        streaming=True,
)
    
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    agent = Agent(model=model, tools=all_tools, system_prompt=SYSTEM_PROMPT)
                                
    start_time = time.time()
    test_queries = [
        "Patient had MI last week",
        "diabetes treatment options", 
        "Patient with diabetes and heart attack needs treatment",
        "DM type 2 with complications",
        "Breast cancer patients with BRCA mutations for clinical trial"
    ]
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n{i}️⃣ Query: '{query}'")
        print("-" * 40)   
        time.sleep(10)                             
        try:
            result = agent(query)
            execution_time = time.time() - start_time
                                            
            print(f"\n⏱️  Execution time: {execution_time:.2f}s")
            print(f"🎯 Response: {result.message['content'][0]['text']}")

        except Exception as e:
            print(f"❌ Error executing agent: {str(e)}")

## Next Steps

Now that you have successfully deployed an MCP server to AgentCore Runtime, you can:

1. **Add More Tools**: Extend your MCP server with additional tools
2. **Custom Authentication**: Implement custom JWT authorizers
3. **Integration**: Integrate with other AgentCore services

## Cleanup (Optional)

If you want to clean up the resources created during this tutorial, run the following cells:

In [ ]:
# print("🗑️  Starting cleanup process...")

# try:
#     ssm_client.delete_parameter(Name='/ols_mcp_server/runtime/agent_arn')
#     print("✓ Parameter Store parameter deleted")
# except ssm_client.exceptions.ParameterNotFound:
#     print("ℹ️  Parameter Store parameter not found")


# print("\n✅ Cleanup completed successfully!")

In [ ]:
# destroy_bedrock_agentcore(
#     config_path=Path(".bedrock_agentcore.yaml"),
#     agent_name=tool_name,
#     delete_ecr_repo=True
# )

# 🎉 Congratulations!

You have successfully:

✅ **Created an MCP server** with custom tools  
✅ **Tested locally** using MCP client  
✅ **Set up authentication** with Amazon Cognito  
✅ **Deployed to AWS** using AgentCore Runtime  
✅ **Invoked remotely** with proper authentication  
✅ **Learned MCP concepts** and best practices  

Your MCP server is now running on Amazon Bedrock AgentCore Runtime and ready for production use!

## Summary

In this tutorial, you learned how to:
- Build MCP servers using FastMCP
- Configure stateless HTTP transport for AgentCore compatibility
- Set up JWT authentication with Amazon Cognito
- Deploy and manage MCP servers on AWS
- Test both locally and remotely
- Use MCP clients for tool invocation

The deployed MCP server can now be integrated into larger AI applications and workflows!